<a href="https://colab.research.google.com/github/PatrickRuan/AI_2025/blob/main/LLM%E5%AF%AB%E5%87%BA_ollamaLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama之取代蔡老師之不易]()

對話中有圖案所以沒有辦法分享：

finally，前後可能花了三十分鐘，
我的目的是demo 讓學員看到只要說清楚，就能夠完成這項任務。
但是你有幾個地方比較可惜，或者說這次的 demo 比較可惜：
1. 這次 demo 中，有一些你沒有辦法自己解決，需要靠有經驗的人介入才解決。
2. 比如： chatgpt 沒有 chatcompletions 很久了，你不知道，給了你 demo code，你也沒有辦法處理。
3. 比如： 我們是處理 ollama 上的 local model 遇到 api_url 時候，你沒有去找出要用 base_url，反而回去套用呼叫 openai api 的 tokens 服務... 你提供的 阿里雲 2025/2/26 的文章就有，但是你只是提出 citation，沒有能夠真的閱讀而找到對的方式。

其他仍舊有一些前後的關係，是人類的介入才處理掉。

### 版本一，驅動 ollama 載入模型，設置模型，用 gradio


In [1]:
!pip install --upgrade openai gradio requests

# 1. 安裝 Ollama（使用官方提供的安裝腳本）
!curl -fsSL https://ollama.com/install.sh | sh

# 2. 下載你需要的模型 gemma3:4b
#!ollama pull gemma3:4b

# 3. 啟動 Ollama 服務，預設會使用 127.0.0.1:11434
!nohup ollama serve > ollama.log 2>&1 &

# 4. 等待 5 秒讓服務啟動
!sleep 5

# 5. 檢查服務日誌
#!cat ollama.log
!ollama pull gemma3:4b

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.9/643.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.70.0
    Uninstalling openai-1.70.0:
      Successfully uninstalled openai-1.70.0
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creat

In [2]:
import openai
import gradio as gr

# 设置 Ollama 的 API 端点
client = openai.OpenAI(
    api_key="test-key",  # Ollama 通常不验证 API 密钥，因此可以填写任意字符串
    base_url="http://127.0.0.1:11434/v1"  # 设置为 Ollama 服务的地址
)

def chatbot(prompt):
    try:
        response = client.chat.completions.create(
            model="gemma3:4b",  # 替换为您在 Ollama 中使用的模型名称
            messages=[
                {"role": "system", "content": "你是一位友善且乐于助人的 AI 助手。"},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"发生错误：{e}"

# 创建 Gradio 界面
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="Ollama 本地模型聊天机器人"
)

# 启动界面
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62489465b1e6745e70.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


進階練習：搭配 ngrok

### 版本二，驅動 ollama 載入模型，設置模型，用 gradio、ngrok

In [4]:
# 安裝必要套件
!pip install --upgrade openai gradio requests flask pyngrok

# 安裝 Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 啟動 Ollama 伺服器
!nohup ollama serve > ollama.log 2>&1 &
!sleep 5

# 拉取 gemma3:4b 模型
!ollama pull gemma3:4b



>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling aeda25e63ebd... 100% ▕▏ 3.3 GB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling b6ae5839783f... 100% ▕▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


In [ ]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok, conf
from getpass import getpass
import openai

# 設定 Ollama OpenAI 介面
client = openai.OpenAI(
    api_key="test-key",  # 可以隨便填
    base_url="http://127.0.0.1:11434/v1"
)

# 建立 Flask app
app = Flask(__name__)

# 輸入 ngrok authtoken
auth_token = getpass("請輸入你的 ngrok authtoken：")
conf.get_default().auth_token = auth_token

# 啟動 ngrok 隧道
public_url = ngrok.connect(5000)
print(f"🌍 Ngrok 公開網址: {public_url}")

# HTML 頁面模板
chat_page = '''
<!DOCTYPE html>
<html>
<head>
    <title>Ollama 聊天機器人</title>
</head>
<body>
    <h2>💬 Ollama Chatbot (gemma3:4b)</h2>
    <form method="POST">
        <textarea name="prompt" rows="4" cols="50" placeholder="輸入你的訊息..."></textarea><br><br>
        <input type="submit" value="送出">
    </form>
    {% if user_prompt %}
        <h4>你問：</h4>
        <p>{{ user_prompt }}</p>
        <h4>Ollama 回答：</h4>
        <p>{{ bot_response }}</p>
    {% endif %}
</body>
</html>
'''

@app.route("/", methods=["GET", "POST"])
def index():
    user_prompt = None
    bot_response = None

    if request.method == "POST":
        user_prompt = request.form.get("prompt")
        if user_prompt:
            try:
                # 呼叫 Ollama 模型
                response = client.chat.completions.create(
                    model="gemma3:4b",
                    messages=[
                        {"role": "system", "content": "你是一位友善且樂於助人的 AI 助手。"},
                        {"role": "user", "content": user_prompt}
                    ]
                )
                bot_response = response.choices[0].message.content.strip()
            except Exception as e:
                bot_response = f"發生錯誤：{e}"

    return render_template_string(chat_page, user_prompt=user_prompt, bot_response=bot_response)

# 啟動 Flask
app.run(host="0.0.0.0", port=5000)



請輸入你的 ngrok authtoken：··········
🌍 Ngrok 公開網址: NgrokTunnel: "https://c11e-35-204-218-138.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:36:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:36:21] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:36:28] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:37:32] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:37:47] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:37:50] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:37:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:37:55] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2025 07:38:02] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Apr/2

# 以下是練功過程的一些失敗...


In [ ]:
import openai
import gradio as gr

# 设置 Ollama 的 API 端点
openai.api_base = "http://127.0.0.1:11434/v1"
openai.api_key = "test-key"  # Ollama 通常不验证 API 密钥，因此可以填写任意字符串

# 定义聊天函数
def chatbot(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gemma3:4b",  # 替换为您在 Ollama 中使用的模型名称
            messages=[
                {"role": "system", "content": "你是一位友善且乐于助人的 AI 助手。"},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message["content"].strip()
    except Exception as e:
        return f"发生错误：{e}"

# 创建 Gradio 界面
iface = gr.Interface(
    fn=chatbot,
    inputs="text",
    outputs="text",
    title="Ollama 本地模型聊天机器人"
)

# 启动界面
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://78b1f0e9cc7536615d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import openai
import gradio as gr
import requests

# 指定 Ollama 的 API 端點（請確保與 Cell 1 使用的埠一致）
openai.api_base = "http://127.0.0.1:11434"
openai.api_key = "test-key"  # Ollama 一般不檢查 API Key，可隨意填寫

# 測試用 requests 呼叫 Ollama 的 /v1/models 端點，確認服務是否正常運作
try:
    r = requests.get("http://127.0.0.1:11434/v1/models")
    print("Models API 回應狀態:", r.status_code)
    print("回應內容:", r.text)
except Exception as e:
    print("測試連線發生錯誤：", e)

# 定義 Chatbot 函式，使用 OpenAI API 介面呼叫 Ollama 模型
def chatbot(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gemma3:4b",
            messages=[{"role": "user", "content": prompt}],
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# 建立 Gradio 介面
iface = gr.Interface(fn=chatbot, inputs="text", outputs="text", title="Ollama Chatbot")
iface.launch(share=True)


Models API 回應狀態: 200
回應內容: {"object":"list","data":null}

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cfd4fb5b20e1635c6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import openai
import gradio as gr

# 建立 openai client，指定 API 金鑰與 Ollama 的 API 端點
client = openai.Client(api_key="test-key", api_base="http://127.0.0.1:11434")

def chatbot(prompt):
    try:
        response = client.chat.completions.create(
            model="gemma3:4b",  # 或是你想使用的其他模型名稱
            messages=[
                {"role": "system", "content": "你是一個友善且樂於助人的 AI 助手。"},
                {"role": "user", "content": prompt}
            ]
        )
        # 回傳回應內容
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# 建立 Gradio 介面
iface = gr.Interface(fn=chatbot, inputs="text", outputs="text", title="Ollama Chatbot")
iface.launch(share=True)


TypeError: OpenAI.__init__() got an unexpected keyword argument 'api_base'

In [ ]:
'''
# 安裝 Ollama（視實際需求與安裝方式調整）
!wget https://example.com/path/to/ollama_installer.sh -O ollama_installer.sh
!chmod +x ollama_installer.sh
!./ollama_installer.sh

# 啟動 Ollama 並載入 gemma3:4b 模型，並在背景執行
!nohup ./ollama start --model gemma3:4b > ollama.log 2>&1 &
# 等待幾秒確保服務已啟動
!sleep 5
'''

--2025-03-23 08:42:01--  https://example.com/path/to/ollama_installer.sh
Resolving example.com (example.com)... 23.192.228.80, 23.192.228.84, 23.215.0.136, ...
Connecting to example.com (example.com)|23.192.228.80|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-23 08:42:01 ERROR 404: Not Found.



In [ ]:
# 使用 curl 安裝 Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# 背景啟動 Ollama 並載入 gemma3:4b 模型，並將 log 輸出到 ollama.log
!nohup ollama start --model gemma3:4b > ollama.log 2>&1 &

# 等待幾秒確保服務啟動
!sleep 5


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
import openai
import gradio as gr

# 指定 OpenAI API 的 base URL 為本機 Ollama 服務
openai.api_base = "http://127.0.0.1:11434"
openai.api_key = "your-api-key"  # 若不需要 API 金鑰，可省略此行

def chatbot(prompt):
    response = openai.ChatCompletion.create(
        model="gemma3:4b",
        messages=[{"role": "user", "content": prompt}],
    )
    return response.choices[0].message.content

# 建立並啟動 Gradio 介面
iface = gr.Interface(fn=chatbot, inputs="text", outputs="text", title="Ollama Chatbot")
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://89647dbf592febf0e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import requests

try:
    r = requests.post(
        "http://127.0.0.1:11434/v1/chat/completions",
        json={
            "model": "gemma3:4b",
            "messages": [{"role": "user", "content": "Hello!"}]
        }
    )
    print(r.status_code, r.text)
except Exception as e:
    print("Error:", e)


Error: HTTPConnectionPool(host='127.0.0.1', port=11434): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x780405ea5a10>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [ ]:
!nohup ollama serve -m gemma3:4b > ollama.log 2>&1 &


In [ ]:
!nohup ollama serve -m gemma3:4b -p 11434 > ollama.log 2>&1 &


In [ ]:
!cat ollama.log


Error: unknown shorthand flag: 'm' in -m


In [ ]:
!ollama pull gemma3:4b


Error: could not connect to ollama app, is it running?
